<a href="https://colab.research.google.com/github/MarySelifanova/PlantLIMEs/blob/master/100321_translocations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Find translocations

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Install and import modules**

In [ ]:
!pip install bio
!pip install XlsxWriter
from Bio import SeqIO
from itertools import combinations
import numpy as np
import re
import pandas as pd
import xlsxwriter

### **Find all possible words of length k in the alignment file**

In [3]:
def find_k_mers(file, k): #file – alignment, k – kmer length
    k_mers = [] #empty list for all possible (not unique) kmers
    
    with open(file, "rU") as handle:  
        for record in SeqIO.parse(handle, "fasta"): #iterating through sequences in the alignment file
            
            test_str = str(record.seq) #test_str – one sequence; res – list of kmers of particular length k
            res = [test_str[x:y] for x, y in combinations(range(len(test_str) + 1), 2) if len(test_str[x:y]) == k and '-' not in test_str[x:y]]
            k_mers.extend(res) #add values 
    
    k_mers_set = set(k_mers) #unique kmers of particular length
    return k_mers_set

### **Create a list of matrices with kmers starts; one matrix for each unique kmer word**

In [10]:
def create_small_matrices(file, k_mers_set): #file – alignment, k_mers_set – unique kmers of particular length
    
    m_list = [] #empty list for dataframes (each dataframe - for unique kmer of particular length)
    k_list = [] #empty list for according kmers 
    num_seq = len([1 for line in open(file) if line.startswith(">")]) #num_seq – number of sequences

    for i in range(len(list(k_mers_set))): #iterating through number of k-mers from the set of particular length

        kmer = list(k_mers_set)[i] #one kmer (one word)
        starts_set = [] #empty kmer's starts list
        starts_list = []

        with open(file, "rU") as handle:

            for record in SeqIO.parse(handle, "fasta"): #iterating through each record in the alignmnet
        
                test_str = str(record.seq) #test_str – one sequence in the alignmnet
                starts = [m.start() for m in re.finditer(kmer, test_str)] #starts list in one sequence
                starts_set.extend(starts)   #add to kmer's starts list  
                starts_list.append(starts)
        
        starts_set = set(starts_set) #no coordinate duplicates in kmer starts in the alignment ...
        starts_set = list(starts_set) #... but still it's a list ...
        starts_set = sorted(starts_set) #... sorted list
        if len(starts_set) > 1: #if there are more than 1 possible start in the kmer
        
          df = pd.DataFrame(np.zeros((num_seq, len(starts_set)))) #df FOR 1 KMER with zeroes, num_col = num unique starts; num_row = num seq in the alignment;
          df.columns = starts_set # rename columns as starts
        
          for s in range(len(starts_list)):  #iterating through number of kmer's starts (list of lists: starts of kmer in each sequence)

            l = starts_list[s] #l – starts list in one sequence
            for j in range(len(l)): #iterating through number of starts in l 
                df.loc[s, l[j]] = 1 #set 1 for this start in df

          df[starts_set] = df[starts_set].apply(pd.to_numeric) #values should be integers (df[starts_set] = df, starts_set - list with colunm names)
          m_list.append(df) #add kmer df to all dfs list
          k_list.append(kmer)

    return m_list, k_list

***m_list – one for each kmer length***

*   *m_list =  = [[df1],[df2],...]*
*   *df1 = [1, 25, ...]*
*   *1, 25, ... – all possible starts (in all seqs of alignment)* of one word

### **Calculate correlations**

In [19]:
def correlations(m_list, k_list, outfile, threshold):  #m_list – see above, outfile – result table, threshold – level of correlation
    
    hit_list = [] #empty list for lists of anticorrelating pairs for each kmer
    for mat in range(len(m_list)): #iterating through matrices for each unique kmer (of the length k)

        l = m_list[mat] #l - matrix for one kmer ('mr')
        corr_matrix = l.corr() #create correlation matrix for one unique kmer (l) of particular size (kmer)
        hits = [(corr_matrix.columns[x], corr_matrix.columns[y], corr_matrix.at[int(corr_matrix.columns[x]),int(corr_matrix.columns[y])]) for x, y in zip(*np.where(corr_matrix.values < - 0.8))] #filter basing on correlation
        kmer = k_list[mat] #corresponding kmer (word), hits – anticorrelating pairs of positions for one kmer ('mr')

        pairs = [] #empty list for anticorrelating pairs
        used = [] #empty list for used cols and rows 
        
        for h in hits: 
            
            col_name = h[1]
            row_name = h[0]
            value = h[2]
            
            if col_name not in used or row_name not in used: #take unique pairs from hits and put them into 'pairs' list
                
                pair = str(kmer) + '_' + str(min(col_name, row_name)) + '_' + str(max(col_name, row_name)) + '_' + str(value) + '_' + str(abs(col_name - row_name)) #forming a pair
                pairs.append(pair) #adding correlated pair 
            
                used.append(col_name)
                used.append(row_name)
        
        hit_list.append(pairs) #append list of anticorrelating pairs of one kmer to a general hit list 
        
    hit_list_checked = [] #create a list of kmer-pos pairs without empty lists (when a kmer doesn't have anticorrelating positions)
    for l in hit_list:
        if l:
            hit_list_checked.append(l)
    return hit_list_checked

### **Create result tables**

In [26]:
def translocations(file, outfile, threshold):

      res_df_list = []
      for k in range(1,5): 
     
          k_mers_set = find_k_mers(file, k) #all posible words of length k in the alignment 'file'
          m_list, k_list = create_small_matrices(file, k_mers_set) #list of matrices (1 m – 1 kmer word; k_list – list of corresponding kmers)
          hit_list_checked = correlations(m_list, k_list, outfile, threshold) #list of 'pairs' lists, where each 'pairs' –  for one kmer

          #create empty lists for columns
          word = []
          pos1 = []
          pos2 = []
          corr = []
          leap = []

          #fill columns with values  
          for l in range(len(hit_list_checked)):
                
              pairs = hit_list_checked[l]  #list of anticorrelating pairs          
              for p in range(len(pairs)): #iterate through pairs 
                    
                  pair = pairs[p].split('_')
                  word.append(pair[0])
                  pos1.append(pair[1])
                  pos2.append(pair[2])
                  corr.append(pair[3])
                  leap.append(pair[4])

          #crate dataframe and fill it with values   
          columns = ['kmer', 'pos1', 'pos2', 'corr', 'leap']
          res_df = pd.DataFrame(columns=columns)
          res_df['kmer'] = word
          res_df['pos1'] = pos1
          res_df['pos2'] = pos2
          res_df['corr'] = corr
          res_df['leap'] = leap

          res_df_list.append(res_df)
      
      # Create a Pandas Excel writer using XlsxWriter as the engine
      writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

      # Write each dataframe to a different worksheet
      res_df_list[0].to_excel(writer, sheet_name='len=1')
      res_df_list[1].to_excel(writer, sheet_name='len=2')
      res_df_list[2].to_excel(writer, sheet_name='len=3')
      res_df_list[3].to_excel(writer, sheet_name='len=4')

      # Close the Pandas Excel writer and output the Excel file
      writer.save()
      
      return res_df_list   

In [27]:
file = '/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str.fasta'
outfile = '/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str_final.xlsx'
threshold = -0.8

res_df_list = translocations(file, outfile, threshold)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: 'U' mode is deprecated
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: 'U' mode is deprecated
  del sys.path[0]


In [28]:
#save res_df_list to file 
import pickle

with open('/content/drive/MyDrive/Translocations/res_df_list.txt', 'wb') as fp:
    pickle.dump(res_df_list, fp)

In [2]:
#and get it back
import pickle

with open ('/content/drive/MyDrive/Translocations/res_df_list.txt', 'rb') as fp:
   rdl  = pickle.load(fp)

# Tune final tables

### **Tune coordinates (position + 1)**

In [6]:
pos_cols = ['pos1', 'pos2']
for d in rdl:
    d['pos1'] = pd.to_numeric(d['pos1']) 
    d['pos2'] = pd.to_numeric(d['pos2'])
    d[pos_cols] += 1

### **Remove duplicates**

In [15]:
print(rdl[0].shape)
print(rdl[1].shape)
print(rdl[2].shape)
print(rdl[3].shape)

(136, 5)
(386, 5)
(39, 5)
(3, 5)


**NGKL > NGKL**

*   NGK > NGK (65-407), GKL > GKL (66-408); 
*   NG > NG(65-407), GK > GK(66-408), KL > KL(67-409)
*   N > N, G > G, K > K, L > L


In [14]:
#subtract kmers (where k = 4) from 1,2,3 -mers
for index4, row4 in rdl[3].iterrows():
    
    p1 = int(row4['pos1'])
    p2 = int(row4['pos2'])
    print(p1, p2)
    
    for index3, row3 in rdl[2].iterrows():

        if int(row3['pos1']) == p1 and int(row3['pos2']) == p2:
            rdl[2].drop(index3, inplace=True)
        if int(row3['pos1']) == p1 + 1 and int(row3['pos2']) == p2 + 1:
            rdl[2].drop(index3, inplace=True)
            
    for index2, row2 in rdl[1].iterrows():

        if int(row2['pos1']) == p1 and int(row2['pos2']) == p2:
          rdl[1].drop(index2, inplace=True)
          print('start1', row2['pos1'], row2['pos2'])

        if int(row2['pos1']) == p1 + 1 and int(row2['pos2']) == p2 + 1:
            rdl[1].drop(index2, inplace=True)
            print('start2', row2['pos1'], row2['pos2'])

        if int(row2['pos1']) == p1 + 2 and int(row2['pos2']) == p2 + 2:
            rdl[1].drop(index2, inplace=True)
            print('start3', row2['pos1'], row2['pos2'])

    for index1, row1 in rdl[0].iterrows():

        if int(row1['pos1']) == p1 and int(row1['pos2']) == p2:
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 1 and int(row1['pos2']) == p2 + 1:
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 2 and int(row1['pos2']) == p2 + 2:
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 3 and int(row1['pos2']) == p2 + 3:
            rdl[0].drop(index1, inplace=True)                          

65 407
409 525
135 551


**NGK > NGK (65-407)** 

*   NG > NG (65-407), GK > GK (66-408)
*   N > N, G > G, K > K

In [ ]:
#subtract kmers (where k = 4) from 1,2,3 -mers
for index3, row3 in rdl[2].iterrows():
    
    p1 = int(row3['pos1'])
    p2 = int(row3['pos2'])
    print(p1, p2)
    
    for index2, row2 in rdl[1].iterrows():

        if int(row2['pos1']) == p1 and int(row2['pos2']) == p2:
          rdl[1].drop(index2, inplace=True)
          print('start1', row2['pos1'], row2['pos2'])

        if int(row2['pos1']) == p1 + 1 and int(row2['pos2']) == p2 + 1:
          rdl[1].drop(index2, inplace=True)
          print('start2', row2['pos1'], row2['pos2'])    


    for index1, row1 in rdl[0].iterrows():

        if int(row1['pos1']) == p1 and int(row1['pos2']) == p2:
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 1 and int(row1['pos2']) == p2 + 1:
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 2 and int(row1['pos2']) == p2 + 2:
            rdl[0].drop(index1, inplace=True)

**NG > NG (65-407)** 

*   N > N (65-407), G > G (66-408)

In [ ]:
#subtract kmers (where k = 2) from 1-mers
for index2, row2 in rdl[1].iterrows():
    
    p1 = int(row2['pos1'])
    p2 = int(row2['pos2'])
    print(p1, p2)

    for index1, row1 in rdl[0].iterrows():
        if int(row1['pos1']) == p1 and int(row1['pos2']):
            rdl[0].drop(index1, inplace=True)
        if int(row1['pos1']) == p1 + 1 and int(row1['pos2']) == p2 + 1:
            rdl[0].drop(index1, inplace=True)

In [12]:
rdl[3]

,kmer,pos1,pos2,corr,leap
0,NGKL,65,407,-0.9923224536590236,342
1,KLNR,409,525,-0.8748574099479502,116
2,VASS,135,551,-0.9783986661292924,416


### **Add translocation statistics (00, 01, 10, 11)**

In [20]:
for i in range(len(rdl)):

  d = rdl[i]
  k = i + 1 

  sum00_l = [] 
  sum01_l = []
  sum10_l = []
  sum11_l = []

  for index, row in d.iterrows():

      p1 = int(row['pos1'])
      p2 = int(row['pos2'])
      w = row['kmer']

      sum00 = 0
      sum01 = 0
      sum10 = 0
      sum11 = 0

      file = '/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str.fasta'
      with open(file, "rU") as handle:

              for record in SeqIO.parse(handle, "fasta"): #iterating through each record in the alignmnet
          
                  test_str = str(record.seq) #test_str – one sequence in the alignmnet
                  test_kmer1 = test_str[p1-1:p1-1+k]
                  test_kmer2 = test_str[p2-1:p2-1+k]

                  if test_kmer1 == w:
                    if test_kmer2 == w:
                      sum11 += 1
                    else:
                      sum10 += 1
                  else:
                    if test_kmer2 == w:
                      sum01 += 1
                    else:
                      sum00 +=1

      sum00_l.append(sum00)
      sum01_l.append(sum01)
      sum10_l.append(sum10)
      sum11_l.append(sum11)

  d['00'] = sum00_l
  d['01'] = sum01_l
  d['10'] = sum10_l
  d['11'] = sum11_l

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: 'U' mode is deprecated


In [ ]:
rdl[2]

In [23]:
outfile = '/content/drive/MyDrive/Translocations/H1N1_H3N2.1557.AA.aligned.1str_tuned.xlsx'
# Create a Pandas Excel writer using XlsxWriter as the engine
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')

# Write each dataframe to a different worksheet
rdl[0].to_excel(writer, sheet_name='len=1')
rdl[1].to_excel(writer, sheet_name='len=2')
rdl[2].to_excel(writer, sheet_name='len=3')
rdl[3].to_excel(writer, sheet_name='len=4')

# Close the Pandas Excel writer and output the Excel file
writer.save()